In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

tcmalloc: large alloc 1073750016 bytes == 0x5b7ba000 @  0x7fa9d09c62a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
0.4.0
True


In [0]:
!pip install --upgrade gensim

    100% |████████████████████████████████| 23.6MB 1.7MB/s 
    100% |████████████████████████████████| 1.4MB 16.0MB/s 
    100% |████████████████████████████████| 133kB 19.9MB/s 
    100% |████████████████████████████████| 61kB 21.0MB/s 
    100% |████████████████████████████████| 4.9MB 6.9MB/s 
    100% |████████████████████████████████| 552kB 24.8MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/Deepbind/ELK1_GM12878_ELK1_(1277-1)_Stanford_AC/"

models_seq_pos_5_1.txt	models_word2vec_model_5_1_60
models_seq_pos_5_2.txt	models_word2vec_model_5_2_60


In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
       process = psutil.Process(os.getpid())
       print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
       print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [0]:
import numpy as np
import csv
import math 
import random
import gzip
from scipy.stats import bernoulli
import torch
from sklearn import metrics
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import gensim
import multiprocessing

In [0]:
nummotif=16 #number of motifs to discover
bases='ACGT' #DNA bases
basesRNA='ACGU'#RNA bases
batch_size=64 #fixed batch size -> see notes to problem about it
dictReverse={'A':'T','C':'G','G':'C','T':'A','N':'N'} #dictionary to implement reverse-complement mode
reverse_mode=False

In [0]:
def dinucshuffle(sequence):
    b=[sequence[i:i+2] for i in range(0, len(sequence), 2)]
    random.shuffle(b)
    d=''.join([str(x) for x in b])
    return d

In [0]:
def complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    complseq = [complement[base] for base in seq]
    return complseq
  
def reverse_complement(seq):
    seq = list(seq)
    seq.reverse()
    return ''.join(complement(seq))


In [0]:
def seq2ngram2(seqs, k, s, dest):
    f = open(seqs)
    lines = f.readlines()
    f.close()

    print('need to n-gram %d lines' % len(lines))
    f = open(dest, 'w')
    for num, line in enumerate(lines):
        if num < 100000:
            line = line[:-1]  # remove '\n' and lower ACGT
            l = len(line)  # length of line

            for i in range(0, l, s):
                if i + k >= l + 1:
                    break

                f.write(''.join(line[i:i + k]))
                f.write(' ')
            f.write('\n')
    f.close()


def word2vect(k, s, vector_dim, root_path, pos_sequences):
    seq2ngram2(pos_sequences, k, s, root_path+'seq_pos_' + str(k) + '_' + str(s) + '.txt')
    sentences = LineSentence(root_path+'seq_pos_' + str(k) + '_' + str(s) + '.txt')

    mode1 = gensim.models.Word2Vec(sentences, iter=20, window=int(12 / s), min_count=0, size=vector_dim,
                                   workers=multiprocessing.cpu_count())
    mode1.save(root_path + 'word2vec_model' + '_' + str(k) + '_' + str(s) + '_' + str(vector_dim))
    word2vec_path=root_path + 'word2vec_model' + '_' + str(k) + '_' + str(s) + '_' + str(vector_dim)
    return word2vec_path

In [0]:
class Chip():
    def __init__(self,filename,pos_seq_dest):
        self.file = filename
        self.pos_seq_dest = pos_seq_dest
            
    def openFile(self):
        train_dataset=[]
        f = open(self.pos_seq_dest, 'w')

        with gzip.open(self.file, 'rt') as data:
            next(data)
            reader = csv.reader(data,delimiter='\t')
           
            for row in reader:
                f.write(row[2])
                f.write('\n')
                f.write(dinucshuffle(row[2]))
                f.write('\n')
                train_dataset.append([row[2],[1]])
                train_dataset.append([dinucshuffle(row[2]),[0]])
                    
        f.close()
        train_dataset_pad=train_dataset

        size=int(len(train_dataset_pad)/3)

        firstvalid=train_dataset_pad[:size]
        secondvalid=train_dataset_pad[size:size+size]
        thirdvalid=train_dataset_pad[size+size:]
        firsttrain=secondvalid+thirdvalid
        secondtrain=firstvalid+thirdvalid
        thirdtrain=firstvalid+secondvalid
        return firsttrain,firstvalid,secondtrain,secondvalid,thirdtrain,thirdvalid,train_dataset_pad
        

In [0]:
class Chip_test():
    def __init__(self,filename):
        self.file = filename
       
            
    def openFile(self):
        test_dataset=[]
        with gzip.open(self.file, 'rt') as data:
            next(data)
            reader = csv.reader(data,delimiter='\t')
            
            for row in reader:
                    test_dataset.append([(row[2]),[int(row[3])]])
            
            
        return test_dataset

In [0]:
# chipseq=Chip('/content/drive/My Drive/Deepbind/ZBTB7A_HepG2_ZBTB7A_(SC-34508)_HudsonAlpha_AC.seq.gz')
dataset_path='/content/drive/My Drive/Deepbind/USF1_HepG2_USF-1_HudsonAlpha_AC.seq.gz'
pos_seq_path='/content/drive/My Drive/Deepbind/USF1_HepG2_USF-1_HudsonAlpha_AC.seq/seq.alldata.txt'
chipseq=Chip(dataset_path,pos_seq_path)



In [0]:
train1,valid1,train2,valid2,train3,valid3,alldataset=chipseq.openFile()

In [0]:
embedding_dim=50
K_mer=5
s=2
word2vec='/content/drive/My Drive/Deepbind/USF1_HepG2_USF-1_HudsonAlpha_AC.seq/models_'
word2vec_path=word2vect(k=K_mer, s=s, vector_dim=embedding_dim, root_path=word2vec,pos_sequences= pos_seq_path)

need to n-gram 42780 lines


In [0]:
model1 = gensim.models.Word2Vec.load(word2vec_path)
weights = torch.FloatTensor(model1.wv.vectors)
print(weights.shape)
print(model1.wv.vocab)

torch.Size([1024, 50])
{'CAGCC': <gensim.models.keyedvectors.Vocab object at 0x7fb05e1e04e0>, 'GCCAT': <gensim.models.keyedvectors.Vocab object at 0x7fb05e1e0ac8>, 'CATTG': <gensim.models.keyedvectors.Vocab object at 0x7fb05e1e0c88>, 'TTGAA': <gensim.models.keyedvectors.Vocab object at 0x7fb05e1e0f98>, 'GAAGA': <gensim.models.keyedvectors.Vocab object at 0x7fb05e1e0710>, 'AGATG': <gensim.models.keyedvectors.Vocab object at 0x7fb05e1e0b38>, 'ATGCC': <gensim.models.keyedvectors.Vocab object at 0x7fb05e1e0d30>, 'GCCCC': <gensim.models.keyedvectors.Vocab object at 0x7fb05e1e0048>, 'CCCAC': <gensim.models.keyedvectors.Vocab object at 0x7fb05e1e02b0>, 'CACTT': <gensim.models.keyedvectors.Vocab object at 0x7fb05e1e0ba8>, 'CTTCA': <gensim.models.keyedvectors.Vocab object at 0x7fb05e1e0f60>, 'TCAGG': <gensim.models.keyedvectors.Vocab object at 0x7fb05e1e00f0>, 'AGGCA': <gensim.models.keyedvectors.Vocab object at 0x7fb05e1e0320>, 'GCACC': <gensim.models.keyedvectors.Vocab object at 0x7fb05e1e082

In [0]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

class chipseq_dataset(Dataset):
    """ Diabetes dataset."""

    def __init__(self,xy=None,wv=None,k=5,s=1):
        self.k=k
        self.s=s
        self.wv=wv
        self.x_data=[el[0] for el in xy]
        self.x_data=self.seqToKmers(self.x_data)
        self.x_data=self.seqToIndices(self.x_data)
#         print(len(self.x_data))
#         print(len(self.x_data[1]))
        self.y_data =np.asarray([el[1] for el in xy ],dtype=np.float32)

        self.x_data =np.asarray(self.x_data,dtype=np.float32)
        self.x_data = torch.LongTensor(self.x_data)
        self.y_data = torch.from_numpy(self.y_data)
        self.len=len(self.x_data)
        
    def seqToIndices(self,data):
        out=[]
        for string_data in data:
          index_data = []
          for word in string_data:
              if word in self.wv:
                  index_data.append(self.wv.vocab[word].index)
              else:
                  print('not found')
          out.append(index_data)
        return out
    def seqToKmers(self,data):
      
        out=[]
        for i in data:
            kmer_list=[]
            for j in range(0,(len(i)-self.k),self.s):
                kmer_list.append(i[j:j+self.k])
            out.append(kmer_list)
        return out


    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len



In [0]:
train1_dataset=chipseq_dataset(train1,model1.wv,K_mer,s=s)
train2_dataset=chipseq_dataset(train2,model1.wv,K_mer,s=s)
train3_dataset=chipseq_dataset(train3,model1.wv,K_mer,s=s)
valid1_dataset=chipseq_dataset(valid1,model1.wv,K_mer,s=s)
valid2_dataset=chipseq_dataset(valid2,model1.wv,K_mer,s=s)
valid3_dataset=chipseq_dataset(valid3,model1.wv,K_mer,s=s)
alldataset_dataset=chipseq_dataset(alldataset,model1.wv,K_mer,s=s)

In [0]:
batchSize=128

train_loader1 = DataLoader(dataset=train1_dataset,batch_size=batchSize,shuffle=True)
train_loader2 = DataLoader(dataset=train2_dataset,batch_size=batchSize,shuffle=True)
train_loader3 = DataLoader(dataset=train3_dataset,batch_size=batchSize,shuffle=True)
valid1_loader = DataLoader(dataset=valid1_dataset,batch_size=batchSize,shuffle=False)
valid2_loader = DataLoader(dataset=valid2_dataset,batch_size=batchSize,shuffle=False)
valid3_loader = DataLoader(dataset=valid3_dataset,batch_size=batchSize,shuffle=False)
alldataset_loader=DataLoader(dataset=alldataset_dataset,batch_size=batchSize,shuffle=True)

train_dataloader=[train_loader1,train_loader2,train_loader3]
valid_dataloader=[valid1_loader,valid2_loader,valid3_loader]


In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
import torch.nn.functional as F
from torch.nn import init




# input of shape(batch_size,inp_chan,iW)
class ConvNet(nn.Module):
    def __init__(self, nummotif,motiflen,dropprob,learning_rate,LSTM_hidden_number,LSTM_hidden_number2,LSTM_layers=1,word2vec_path=None,embedding_dim=50):
      
        super(ConvNet, self).__init__()
        
        self.dropprob=dropprob
        self.learning_rate=learning_rate
        self.LSTM_hidden_number=LSTM_hidden_number
        self.LSTM_layers=LSTM_layers
        
        model_wv = gensim.models.Word2Vec.load(word2vec_path)
        weights = torch.FloatTensor(model_wv.wv.vectors)
        self.embedding = nn.Embedding.from_pretrained(weights,freeze=False)
        
        self.wConv=torch.randn(nummotif,embedding_dim,motiflen).to(device)
        torch.nn.init.xavier_uniform(self.wConv)
        self.wConv.requires_grad=True

        self.convBias=torch.randn(nummotif).to(device)
        torch.nn.init.normal_(self.convBias)
        self.convBias.requires_grad=True
        
        self.lstm1=nn.LSTM(nummotif, LSTM_hidden_number,num_layers=1,batch_first=True)
        if LSTM_layers>1:
            self.lstm2=nn.LSTM(LSTM_hidden_number, LSTM_hidden_number2,num_layers=1,batch_first=True)
            for layer_p in self.lstm2._all_weights:
              for p in layer_p:
                  if 'weight' in p:
                      init.normal(self.lstm2.__getattr__(p), 0.0, 1e-3)
        for layer_p in self.lstm1._all_weights:
            for p in layer_p:
                if 'weight' in p:
                    init.xavier_uniform(self.lstm1.__getattr__(p))
        if LSTM_layers>1:
          
            self.fc1 = nn.Linear(LSTM_hidden_number2, 20) 
        else:
          self.fc1 = nn.Linear(LSTM_hidden_number, 20)

        torch.nn.init.xavier_uniform(self.fc1.weight)
        self.fc1.bias.data.fill_(0.01)
        
        self.fc2 = nn.Linear(20, 1)
        torch.nn.init.xavier_uniform(self.fc2.weight)
        self.fc2.bias.data.fill_(0.01)
        
        
        self.drop1=nn.Dropout(p=dropprob)
        self.drop2=nn.Dropout(p=0.9)
    
    def forward(self, x):
        x=x.long() 
        out=self.embedding(x)
        out=out.permute(0, 2, 1)
#         print(x.shape)
        out=F.conv1d(out, self.wConv, bias=self.convBias, stride=1, padding=0)
#         print(out.shape)
        out=out.clamp(min=0)
#         print(out.shape)
#         maxPool, _ = torch.max(out, dim=2)
#         out=torch.unsqueeze(maxPool,2)


        ## shape(Batch,Channels,seq_len)
#         print(out.shape)
        out=out.permute(0, 2, 1)
        ## shape((batch, seq, feature)
#         print(out.shape)
        output, (hn, cn)=self.lstm1(out)
        if self.LSTM_layers==2:
             output, (hn, cn)=self.lstm2(output)
        hn=torch.squeeze(hn, 0)
        # from (1,batch,hidden_size) to (batch,hidden_size)
#         hn=hn.view(-1,self.LSTM_hidden_number)
#         print(hn.shape)
        out=self.drop1(hn)
        out = self.fc1(out)
        out=out.clamp(min=0)
#         out=self.drop2(out)
        out = self.fc2(out)
        return out
      
      
      
      
      
      

In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
import torch.nn.functional as F
from torch.nn import init




# input of shape(batch_size,inp_chan,iW)
class ConvNet_b(nn.Module):
    def __init__(self, nummotif,motiflen,dropprob,learning_rate,LSTM_hidden_number,LSTM_hidden_number2,LSTM_layers=1,word2vec_path=None,embedding_dim=50):
      
        super(ConvNet_b, self).__init__()
        
        self.dropprob=dropprob
        self.learning_rate=learning_rate
        self.LSTM_hidden_number=LSTM_hidden_number
        self.LSTM_layers=LSTM_layers
        
        model_wv = gensim.models.Word2Vec.load(word2vec_path)
        weights = torch.FloatTensor(model_wv.wv.vectors)
        self.embedding = nn.Embedding.from_pretrained(weights,freeze=False)
        
        self.wConv=torch.randn(nummotif,embedding_dim,motiflen).to(device)
        torch.nn.init.xavier_uniform(self.wConv)
        self.wConv.requires_grad=True

        self.convBias=torch.randn(nummotif).to(device)
        torch.nn.init.normal_(self.convBias)
        self.convBias.requires_grad=True
        
        self.GRU=nn.GRU(nummotif, LSTM_hidden_number,num_layers=1,batch_first=True,bidirectional=True)
        if LSTM_layers>1:
            self.lstm2=nn.LSTM(LSTM_hidden_number, LSTM_hidden_number2,num_layers=1,batch_first=True)
            for layer_p in self.lstm2._all_weights:
              for p in layer_p:
                  if 'weight' in p:
                      init.normal(self.lstm2.__getattr__(p), 0.0, 1e-3)
        for layer_p in self.GRU._all_weights:
            for p in layer_p:
                if 'weight' in p:
                    init.xavier_uniform(self.GRU.__getattr__(p))
        if LSTM_layers>1:
          
            self.fc1 = nn.Linear(LSTM_hidden_number2, 20) 
        else:
            self.fc1 = nn.Linear(2*LSTM_hidden_number, 1)

        torch.nn.init.xavier_uniform(self.fc1.weight)
        self.fc1.bias.data.fill_(0.01)
        
        self.fc2 = nn.Linear(20, 1)
        torch.nn.init.xavier_uniform(self.fc2.weight)
        self.fc2.bias.data.fill_(0.01)
        
        
        self.drop1=nn.Dropout(p=dropprob)
        self.drop2=nn.Dropout(p=0.9)
    
    def forward(self, x):
        x=x.long() 
        out=self.embedding(x)
        out=out.permute(0, 2, 1)
        out=F.conv1d(out, self.wConv, bias=self.convBias, stride=1, padding=0)
        out=out.clamp(min=0)

        ## shape(Batch,Channels,seq_len)
        out=out.permute(0, 2, 1)
        ## shape((batch, seq, feature)
        output, hn=self.GRU(out)
        Normal_LSTM=hn[0, :, :]
        Rev_LSTM=hn[1, :, :]
        
        Concat_LSTM = torch.cat((Normal_LSTM, Rev_LSTM), 1)
 
      
        
        if self.LSTM_layers==2:
             output, (hn, cn)=self.lstm2(output)

        out=self.drop1(Concat_LSTM)
        out = self.fc1(out)
        return out
      
      
      
      
      
      

In [0]:
best_AUC=0
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
# device='cpu'

for number in range(5):

    dropoutList=[0.5,0.7,1]         
    dropprob=random.choice(dropoutList)
    
    learning_rateList=[1e-2,1e-3,1e-4]
    learning_rate=random.choice(learning_rateList)
 
    weight_decayList=[1e-5,0]
    weightDecay=random.choice(weight_decayList)
  
    LSTM_hidden_number=15
    LSTM_hidden_number2=10
    model_auc=[[],[],[]]
    n=0
    for learning_rate in [1e-2,1e-3,1e-4]:
       for LSTM_hidden_number in [10,15,20,30]:
        for weightDecay in [0.02,0.01,0.001]:
          for nummotif in [10,20,30]:
            for motiflen in [7,10,15,20]:
              
              n+=1
              for kk in range(3):
                  model = ConvNet_b(nummotif,motiflen,0.5,learning_rate,LSTM_hidden_number,LSTM_hidden_number2,LSTM_layers=1,word2vec_path=word2vec_path,embedding_dim=embedding_dim).to(device)
                  model.train()
                  optimizer = torch.optim.SGD([params for params in model.parameters()]+[model.wConv,model.convBias], lr=model.learning_rate,weight_decay=weightDecay)

                  train_loader=train_dataloader[kk]
                  valid_loader=valid_dataloader[kk]
                  for epoch in range(40):
                      model.train()
                      auc=[]
                      for i, (data, target) in enumerate(train_loader):
                          data = data.to(device)
                          target = target.to(device)


                          # Forward pass
                          output = model(data)

                          loss = F.binary_cross_entropy(torch.sigmoid(output),target)


                          optimizer.zero_grad()
                          loss.backward()
                          optimizer.step()


                      with torch.no_grad():
                          model.eval()
                          auc=[]
                          for i, (data, target) in enumerate(train_loader):
                              data = data.to(device)
                              target = target.to(device)

                              # Forward pass
                              output = model(data)
                              pred_sig=torch.sigmoid(output)
                              pred=pred_sig.cpu().detach().numpy().reshape(output.shape[0])
                              labels=target.cpu().numpy().reshape(output.shape[0])

                              auc.append(metrics.roc_auc_score(labels, pred))
              #                         print(np.mean(auc))
                          model_auc[kk].append(np.mean(auc))
                          print('AUC performance on the training data',kk+1,' = ',np.mean(auc))


                  with torch.no_grad():
                      model.eval()
                      auc=[]
                      for i, (data, target) in enumerate(valid_loader):
                          data = data.to(device)
                          target = target.to(device)

                          # Forward pass
                          output = model(data)
                          pred_sig=torch.sigmoid(output)
                          pred=pred_sig.cpu().detach().numpy().reshape(output.shape[0])
                          labels=target.cpu().numpy().reshape(output.shape[0])

                          auc.append(metrics.roc_auc_score(labels, pred))

                      model_auc[kk].append(np.mean(auc))
              AUC=(model_auc[0][n]+model_auc[1][n]+model_auc[2][n])/3
              if AUC>best_AUC:
                  best_AUC=AUC
                  print('best AUC= ',AUC)
                  print('best parameters: ')
                  print(learning_rate,' ',LSTM_hidden_number,' ',weightDecay,' ',nummotif,' ',motiflen)

                  print('                   ##########################################               ')

            
 

cuda:0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


AUC performance on the training data 1  =  0.5252159741348053
AUC performance on the training data 1  =  0.5492179812615664
AUC performance on the training data 1  =  0.568721009193198
AUC performance on the training data 1  =  0.5859581946071115
AUC performance on the training data 1  =  0.6006771774580181
AUC performance on the training data 1  =  0.6129243907733376
AUC performance on the training data 1  =  0.6255090858794713
AUC performance on the training data 1  =  0.637198199206143
AUC performance on the training data 1  =  0.6476328682544087
AUC performance on the training data 1  =  0.6587557820517485
AUC performance on the training data 1  =  0.6680848875178798
AUC performance on the training data 1  =  0.6776179007097027
AUC performance on the training data 1  =  0.6866340117314925
AUC performance on the training data 1  =  0.6935616747495289
AUC performance on the training data 1  =  0.7023066690613031
AUC performance on the training data 1  =  0.7106137393923206
AUC perfor

KeyboardInterrupt: ignored

In [0]:
chipseq_test=Chip_test('/content/drive/My Drive/Deepbind/USF1_HepG2_USF-1_HudsonAlpha_B.seq.gz')
test_data=chipseq_test.openFile()
print(len(test_data))
test_dataset=chipseq_dataset(test_data,model1.wv,K_mer,s=s)
batchSize=test_dataset.__len__()
test_loader = DataLoader(dataset=test_dataset,batch_size=batchSize,shuffle=False)

1000


In [0]:
learning_rate=0.015
LSTM_hidden_number=10
weightDecay=0.022
nummotif=12
motiflen=14
model = ConvNet_b(nummotif,motiflen,0.5,learning_rate,LSTM_hidden_number,10,LSTM_layers=1,word2vec_path=word2vec_path,embedding_dim=embedding_dim).to(device)
model.train()
optimizer = torch.optim.SGD([params for params in model.parameters()]+[model.wConv,model.convBias], lr=model.learning_rate,weight_decay=weightDecay)

train_loader=alldataset_loader
valid_loader=test_loader
for epoch in range(40):
    model.train()
  
    
    for i, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)


        # Forward pass
        output = model(data)

        loss = F.binary_cross_entropy(torch.sigmoid(output),target)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()






with torch.no_grad():
    model.eval()
    auc=[]
    for i, (data, target) in enumerate(valid_loader):
        data = data.to(device)
        target = target.to(device)

        # Forward pass
        output = model(data)
        pred_sig=torch.sigmoid(output)
        pred=pred_sig.cpu().detach().numpy().reshape(output.shape[0])
        labels=target.cpu().numpy().reshape(output.shape[0])

        auc.append(metrics.roc_auc_score(labels, pred))


    print('AUC performance on test data number  = ',str(test_loader),' ',np.mean(auc))



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


AUC performance on test data number  =  <torch.utils.data.dataloader.DataLoader object at 0x7fbb9b222320>   0.9842040000000001
